# BLIBIOTECAS

In [4]:
from pathlib import Path
import progressbar as pb # Visualizar barra de progresso, bom para operações de longa duração
import pandas as pd
import numpy as np
import random
import warnings # supress warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity
# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format

# Exercício Prático - Matrizes Esparsas

# 1. Carregar os datasets disponibilizados pela DNC ou fazer download via Kaggle dos datasets 'arashnic/book-recommendation-dataset'

In [5]:
%env KAGGLE_USERNAME = tomasmenegatto
%env KAGGLE_KEY = 11458e6485cef3e20517783268b9afac

!content/book-recommendation-dataset
!kaggle datasets download -d arashnic/book-recommendation-dataset --unzip -p content/book-recommendation-dataset
!ls content/book-recommendation-dataset

DATA_PATH = Path("content/book-recommendation-dataset")

book_df = pd.read_csv(DATA_PATH / 'Books.csv')
ratings_df = pd.read_csv(DATA_PATH / 'Ratings.csv')
users_df = pd.read_csv(DATA_PATH / 'Users.csv')

env: KAGGLE_USERNAME=tomasmenegatto
env: KAGGLE_KEY=11458e6485cef3e20517783268b9afac
/bin/bash: content/book-recommendation-dataset: Is a directory
 38% 9.00M/23.8M [00:00<00:00, 45.7MB/s]
100% 23.8M/23.8M [00:00<00:00, 79.0MB/s]
Books.csv  Ratings.csv	Users.csv


# 2. Amostrar e obter a matriz de relacionamento entre os clientes e os livros.


##2.1. Matriz de Similaridade

In [6]:
# Faz uma cópia para que o dataset original não seja alterado
def start_pipeline(dataf):
    return dataf.copy()

# Agrupa o dateset por usuários e conta quantos livros diferentes cada um tem
# Ordena para que os usuários com mais livros fiquem no topo do dataset
def get_user_counts(dataf): 
  return dataf.groupby(by='User-ID', as_index=False) \
    .agg({'ISBN': pd.Series.nunique}) \
    .rename(columns={'ISBN': 'ISBN_count'}) \
    .set_index('User-ID') \
    .sort_values('ISBN_count', ascending = False)

# Agrupa o dateset por livros e conta quantos usuários diferentes cada lirvro tem
# Ordena para que os livros com mais usuários diferentes fiquem no topo do dataset
def get_book_counts(dataf): 
  return dataf.groupby(by='ISBN', as_index=False) \
    .agg({'User-ID': pd.Series.nunique}) \
    .rename(columns={'User-ID': 'User-ID_count'}) \
    .set_index('ISBN') \
    .sort_values('User-ID_count', ascending = False)

#Recebe o dataset e o tamanho da amostra
 # Retorna as n primeiras linhas do dataset recebido em formato de lista
def get_n_top_values(dataf, n): 
  return dataf.head(sample_size) \
    .index \
    .tolist()

 # Executa as funções acima para rankear os usuários
sample_size = 1000
top_user_ids = ratings_df \
  .pipe(start_pipeline) \
  .pipe(get_user_counts) \
  .pipe(get_n_top_values, sample_size)

# Executa as funções acima para rankear os livros
sample_size = 999  
top_book_ids = ratings_df \
  .pipe(start_pipeline) \
  .pipe(get_book_counts) \
  .pipe(get_n_top_values, sample_size)

##2.2. Matriz de relacionamento entre clientes

In [7]:
#Transforma a amostra em um dataset, renomeando as colunas.
def transform_sample(dataf):
  return pd.DataFrame({
      "user": dataf["User-ID"],
      "book": dataf["ISBN"],
      "action": dataf["Book-Rating"]
  })

# Seleciona só os User-IDs que estão entre os top Users, transforma amostra em um dataframe e organiza o dataset em uma matriz Cliente x Produto 
matriz_de_relacionamento_user_sampled = ratings_df[ratings_df["User-ID"].isin(top_user_ids)] \
  .pipe(transform_sample) \
  .pivot_table(index="user", columns="book", values="action").fillna(0)

In [8]:
# Verifica a dimensão da matriz C x P - 1000 Clientes (tamanho da amostra escolhida) e 212.171 Produtos (livros)
matriz_de_relacionamento_user_sampled.shape

(1000, 212171)

In [9]:
matriz_de_relacionamento_user_sampled.head(10)

book,0330299891,0375404120,0586045007,9022906116,9032803328,9044922564,9044922572,9044922718,9044923161,904492401X,9061002273,*0452281903,*0515128325,+0451197399,0 00 612183 7,0 00 614494 2,0 7336 1053 6,0 907 062 008,0*708880258,0.330241664,00000000,000000000,0000000000,00000000000,000000000000,0000000000000,0000000029841,0000000030670,0000000033220,0000000051,0000001042283,0000001481,0000018030,0000722105029,0000913154,0001010565,0001046438,000104687X,0001047213,0001047639,...,O42516313X,O439060737,O439161673,O446611638,O451203771,O499139921,O590418262,O590477412,O59052528X,O67174142X,O774037474,O77O428452,O786001690,O805063196,O856856428,O861366301,O880015152,O889950962,O9088446X,PS3569E84K5,Q380708353,Q912838027,ROO48580353,SBN67001026X,T110105305048,T110105705048,TBR0385495641,THEALLTRUETRA,THECATASTROPH,THEFLYINGACE,UNGRANDHOMMED,X000000000,"YOUTELLEM,AND",ZR903CX0003,"\0432534220\""""","\2842053052\""""",b00005wz75,cn108465,cn113107,Ô½crosoft
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
254,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2033,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2276,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2766,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2977,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3363,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3757,0.000,0.000,

##2.3. Matriz de relacionamento entre os livros

In [10]:
# Seleciona só os Books que estão entre os top books, transforma a amostra em um dataframe e organiza o dataset em uma matriz Cliente x Produto 
matriz_de_relacionamento_book_sampled = ratings_df[ratings_df["ISBN"].isin(top_book_ids)] \
  .pipe(transform_sample) \
  .pivot_table(index="user", columns="book", values="action").fillna(0)

In [11]:
# Verifica as dimensões da matriz CxP transposta, ou seja: PxC
matriz_de_relacionamento_book_sampled.T.shape

(999, 35612)

In [12]:
matriz_de_relacionamento_book_sampled.T.head(10)

user,9,14,16,17,26,39,42,44,51,67,75,91,95,97,99,114,125,132,135,144,165,176,178,185,193,224,232,242,243,244,254,256,280,332,367,383,384,387,388,408,...,278545,278550,278551,278552,278554,278556,278561,278563,278582,278586,278633,278645,278653,278663,278672,278692,278694,278698,278723,278724,278732,278740,278755,278769,278771,278773,278774,278781,278798,278800,278807,278813,278819,278828,278832,278836,278843,278844,278851,278854
book,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000649840X,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
002542730X,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0060008032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0060096195,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,9.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
006016848X,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0060173289,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.000,0.000,0.000,0.000
0060175400,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0060188731,0.000

# 3. Obter a matriz de relacionamento ESPARSA entre os clientes e os livros

In [13]:
# Para obter a matriz esparsa basta dummificar as colunas User-ID e Books do dataset originial

matriz_relacionamento_esparsa = pd.get_dummies(ratings_df.sample(1000), columns=["User-ID", "ISBN"], sparse=True)

matriz_relacionamento_esparsa

# Se tentássemos fazer o mesmo para matriz não esparsa, não teríamos memória suficiente

,Book-Rating,User-ID_183,User-ID_243,User-ID_1343,User-ID_1435,User-ID_2132,User-ID_3326,User-ID_3363,User-ID_3757,User-ID_4017,User-ID_5452,User-ID_5476,User-ID_5697,User-ID_6251,User-ID_6543,User-ID_6577,User-ID_6603,User-ID_6662,User-ID_7158,User-ID_7312,User-ID_7346,User-ID_7623,User-ID_8019,User-ID_8284,User-ID_8713,User-ID_9138,User-ID_9183,User-ID_9440,User-ID_9516,User-ID_9525,User-ID_9856,User-ID_9992,User-ID_10653,User-ID_11676,User-ID_12982,User-ID_13131,User-ID_13136,User-ID_13668,User-ID_13935,User-ID_14404,...,ISBN_3893311661,ISBN_3893971955,ISBN_393299244X,ISBN_685341176,ISBN_8402068820,ISBN_8402076084,ISBN_8426321720,ISBN_8427285809,ISBN_8429749454,ISBN_8430095861,ISBN_8431629274,ISBN_8432222143,ISBN_8433920863,ISBN_8434814609,ISBN_8435015068,ISBN_8440633408,ISBN_8473200038,ISBN_847432081X,ISBN_8482381393,ISBN_8497593715,ISBN_8572323651,ISBN_8804314907,ISBN_8806143042,ISBN_8807701170,ISBN_8807814331,ISBN_8842912034,ISBN_8845202011,ISBN_884541096X,ISBN_8873059023,ISBN_9020401173,ISBN_902546971X,ISBN_9074336671,ISBN_949756016,ISBN_9511125044,ISBN_9511154168,ISBN_9684443102,ISBN_972412102X,ISBN_9724128725,ISBN_9724722678,ISBN_9788479790738
902471,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1025125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
434689,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
453573,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1112797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
401203,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
218755,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
812814,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#4. Obter a matriz de similaridade (via cosine_similarity) entre livros com a matriz NÃO ESPARSA.

##4.1. Para Clientes

In [14]:
# Calculando a similaridade entre os usuários através da multiplicação da matriz CxP pela sua transposta
cos_user = cosine_similarity(matriz_de_relacionamento_user_sampled)
#Imprimindo o resultado
print(cos_user)

[[1.         0.0528255  0.         ... 0.         0.         0.        ]
 [0.0528255  1.         0.04675324 ... 0.         0.         0.        ]
 [0.         0.04675324 1.         ... 0.02334216 0.         0.        ]
 ...
 [0.         0.         0.02334216 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


##4.2. Para Livros

In [15]:
# Calculando a similaridade entre os livros através da multiplicação da matriz Produto x Cliente pela sua transposta 
cos_item = cosine_similarity(matriz_de_relacionamento_book_sampled.T)
# Imprimindo o resultado
print(cos_item)

[[1.         0.05983705 0.02857749 ... 0.02202635 0.01551346 0.        ]
 [0.05983705 1.         0.01555774 ... 0.02620312 0.         0.        ]
 [0.02857749 0.01555774 1.         ... 0.04822782 0.         0.        ]
 ...
 [0.02202635 0.02620312 0.04822782 ... 1.         0.         0.        ]
 [0.01551346 0.         0.         ... 0.         1.         0.03301185]
 [0.         0.         0.         ... 0.         0.03301185 1.        ]]


#5. Obter a matriz de similaridade (via multiplicação de matrizes) entre livros com a matriz ESPARSA.

In [19]:
matriz_similaridade_esparsa = matriz_relacionamento_esparsa.T.multiply(matriz_relacionamento_esparsa)
matriz_similaridade_esparsa

,1831,3963,4165,6206,6879,7586,7858,9344,9814,9972,12638,12945,14629,17663,18663,19949,20835,23599,23661,24374,25720,27730,28943,28950,29007,29126,30698,31403,31962,32913,33746,35068,36544,38226,38458,39149,39150,39231,39243,39888,...,User-ID_9138,User-ID_91390,User-ID_91449,User-ID_91576,User-ID_91647,User-ID_9183,User-ID_92696,User-ID_92810,User-ID_92853,User-ID_92861,User-ID_93047,User-ID_93147,User-ID_93420,User-ID_94242,User-ID_94347,User-ID_9440,User-ID_94547,User-ID_94853,User-ID_94985,User-ID_9516,User-ID_9525,User-ID_95902,User-ID_95907,User-ID_96482,User-ID_96493,User-ID_96528,User-ID_97368,User-ID_98230,User-ID_98255,User-ID_98297,User-ID_98391,User-ID_98440,User-ID_98468,User-ID_9856,User-ID_98723,User-ID_98741,User-ID_99032,User-ID_99227,User-ID_9992,User-ID_99951
1831,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3963,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4165,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6206,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6879,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
User-ID_98741,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
User-ID_99032,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
User-ID_99227,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
User-ID_9992,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

#6. Qual a diferença de tamanhos das matrizes?

In [20]:
# Criando função para mensurar a quantidade de memória utilizada
def print_memory_usage_of_data_frame(df, bytes_to_mb_div = 0.000001):
    mem = round(df.memory_usage().sum() * bytes_to_mb_div, 3) 
    print("Memory usage is " + str(mem) + " MB")

In [21]:
# Exibindo o tempo de processamento
%%time

print_memory_usage_of_data_frame(matriz_de_relacionamento_book_sampled)

Memory usage is 286.207 MB
CPU times: user 10.9 ms, sys: 0 ns, total: 10.9 ms
Wall time: 12.5 ms


In [22]:
# Exibindo o tempo de processamento
%%time

print_memory_usage_of_data_frame(matriz_similaridade_esparsa)

Memory usage is 64.411 MB
CPU times: user 89.1 ms, sys: 2.75 ms, total: 91.9 ms
Wall time: 92.9 ms


#**RESPOSTA:** Enquanto a matriz densa ocupa 286.207 MB, a matriz esparsa tem apenas 64.411 MB (22% da matriz densa)